In [16]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [17]:
#for databricks
import pandas as pd
import xml.etree.ElementTree as ET
from sqlalchemy import create_engine
import os
from os import listdir
from os.path import isfile, join
import re
import zipfile
import pyspark
from pyspark.sql import SparkSession


In [18]:
#Create SparkSession
spark = SparkSession.builder.master("local[1]").appName("data-gurus").getOrCreate()

In [19]:

#file 0
def AnlagenEegSolar_columns():
    my_dict = {'EegMaStRNummer': [],
               'Registrierungsdatum': [],
               'Zuschlagsnummer': [],
               'MieterstromErsteZuordnungZuschlag': [],
               'InstallierteLeistung': [],
               'AusschreibungZuschlag': [],
               'AnlagenkennzifferAnlagenregister_nv': [],
               'ZugeordneteGebotsmenge': [],
               'AnlageBetriebsstatus': [],
               'DatumLetzteAktualisierung': [],
               'RegistrierungsnummerPvMeldeportal_nv': [],
               'VerknuepfteEinheitenMaStRNummern': [],
               'MieterstromMeldedatum': [],
               'EegInbetriebnahmedatum': [],
               'AnlagenschluesselEeg': [],
               'InanspruchnahmeZahlungNachEeg': [],
               'AnlagenkennzifferAnlagenregister': [],
               'MieterstromZugeordnet': [],
               'RegistrierungsnummerPvMeldeportal': []}
    return my_dict

def AnlagenEegSolar_datatypes_change(df):

    # df['EegMaStRNummer'] = pd.to_str(df['EegMaStRNummer'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])
    # df['Zuschlagsnummer'] = pd.to_datetime(df['Zuschlagsnummer'])
    df['MieterstromErsteZuordnungZuschlag'] = pd.to_datetime(df['MieterstromErsteZuordnungZuschlag'])
    df['InstallierteLeistung'] = pd.to_numeric(df['InstallierteLeistung'], errors='coerce')
    df['AusschreibungZuschlag'] = pd.to_numeric(df['AusschreibungZuschlag'], errors='coerce')
    df['AnlagenkennzifferAnlagenregister_nv'] = pd.to_numeric(df['AnlagenkennzifferAnlagenregister_nv'])
    df['ZugeordneteGebotsmenge'] = pd.to_numeric(df['ZugeordneteGebotsmenge'], errors='coerce')
    df['AnlageBetriebsstatus'] = pd.to_numeric(df['AnlageBetriebsstatus'], errors='coerce')
    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])
    df['RegistrierungsnummerPvMeldeportal_nv'] = pd.to_numeric(df['RegistrierungsnummerPvMeldeportal_nv'],
                                                               errors='coerce')
    # df['VerknuepfteEinheitenMaStRNummern'] = pd.to_str(df['VerknuepfteEinheitenMaStRNummern'])
    df['MieterstromMeldedatum'] = pd.to_datetime(df['MieterstromMeldedatum'])
    df['EegInbetriebnahmedatum'] = pd.to_datetime(df['EegInbetriebnahmedatum'])
    # df['AnlagenschluesselEeg'] = pd.to_str(df['AnlagenschluesselEeg'])
    df['InanspruchnahmeZahlungNachEeg'] = pd.to_numeric(df['InanspruchnahmeZahlungNachEeg'], errors='coerce')
    # df['AnlagenkennzifferAnlagenregister'] = pd.to_str(df['AnlagenkennzifferAnlagenregister'])
    df['MieterstromZugeordnet'] = pd.to_numeric(df['MieterstromZugeordnet'], errors='coerce')
    # df['RegistrierungsnummerPvMeldeportal'] = pd.to_str(df['RegistrierungsnummerPvMeldeportal'])


In [20]:

#file 1
def AnlagenEegSpeicher_columns():

    my_dict={'EegMaStRNummer':[],
             'Registrierungsdatum':[],
             'EegInbetriebnahmedatum':[],
             'VerknuepfteEinheitenMaStRNummern':[],
             'DatumLetzteAktualisierung':[]
             }
    return my_dict

def AnlagenEegSpeicher_datatypes_change(df):

    #df['EegMaStRNummer'] = pd.to_datetime(df['EegMaStRNummer'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])
    df['EegInbetriebnahmedatum'] = pd.to_datetime(df['EegInbetriebnahmedatum'])
    #df['VerknuepfteEinheitenMaStRNummern'] = pd.to_datetime(df['VerknuepfteEinheitenMaStRNummern'])
    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])


In [21]:

#file 2
def AnlagenStromSpeicher_columns():

    my_dict ={'MaStRNummer':[],
              'DatumLetzteAktualisierung':[],
              'NutzbareSpeicherkapazitaet':[],
              'AnlageBetriebsstatus':[],
              'VerknuepfteEinheitenMaStRNummern':[],
              'Registrierungsdatum':[]}
    return my_dict

def AnlagenStromSpeicher_datatypes_change(df):
    df['AnlageBetriebsstatus'] = pd.to_numeric(df['AnlageBetriebsstatus'])
    df['NutzbareSpeicherkapazitaet'] = pd.to_numeric(df['NutzbareSpeicherkapazitaet'])
    # df['VerknuepfteEinheitenMaStRNummern'] = pd.to_datetime(df['VerknuepfteEinheitenMaStRNummern'])
    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])
    # df['MaStRNummer'] = pd.to_datetime(df['MaStRNummer'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])


In [22]:

#file 3
def EinheitenStromSpeicher_columns():

    my_dict={'DatumLetzteAktualisierung':[],
             'GeplantesInbetriebnahmedatum':[],
             'Energietraeger':[],
             'NameStromerzeugungseinheit':[],
             'Ort':[],
             'Notstromaggregat':[],
             'Breitengrad':[],
             'Kraftwerksnummer_nv':[],
             'Registrierungsdatum':[],
             'Gemeindeschluessel':[],
             'BestandteilGrenzkraftwerk':[],
             'AnlagenbetreiberMastrNummer':[],
             'ZugeordnenteWirkleistungWechselrichter':[],
             'Land':[],
             'WeicDisplayName':[],
             'AcDcKoppelung':[],
             'Laengengrad':[],
             'PumpbetriebKontinuierlichRegelbar':[],
             'GenMastrNummer':[],
             'Gemarkung':[],
             'EinheitSystemstatus':[],
             'EinheitBetriebsstatus':[],
             'Einspeisungsart':[],
             'Hausnummer_nv':[],
             'Postleitzahl':[],
             'NichtVorhandenInMigriertenEinheiten':[],
             'Technologie':[],
             'Kraftwerksnummer':[],
             'Pumpspeichertechnologie':[],
             'Batterietechnologie':[],
             'FernsteuerbarkeitDv':[],
             'NetzbetreiberpruefungStatus':[],
             'DatumEndgueltigeStilllegung':[],
             'DatumDesBetreiberwechsels':[],
             'FernsteuerbarkeitDr':[],
             'DatumBeginnVoruebergehendeStilllegung':[],
             'NetzbetreiberpruefungDatum':[],
             'Weic':[],
             'EegMaStRNummer':[],
             'SpeMastrNummer':[],
             'DatumRegistrierungDesBetreiberwechsels':[],
             'Bundesland':[],
             'AnschlussAnHoechstOderHochSpannung':[],
             'Einsatzort':[],
             'Einsatzverantwortlicher':[],
             'PumpbetriebLeistungsaufnahme':[],
             'EegAnlagentyp':[],
             'LokationMaStRNummer':[],
             'FlurFlurstuecknummern':[],
             'Inbetriebnahmedatum':[],
             'StrasseNichtGefunden':[],
             'Bruttoleistung':[],
             'Adresszusatz':[],
             'Weic_nv':[],
             'Gemeinde':[],
             'Landkreis':[],
             'HausnummerNichtGefunden':[],
             'EinheitMastrNummer':[],
             'Nettonennleistung':[],
             'Hausnummer':[],
             'FernsteuerbarkeitNb':[],
             'DatumWiederaufnahmeBetrieb':[],
             'Strasse':[],
             'NettonennleistungDeutschland':[]}
    return my_dict

def EinheitenStromSpeicher_datatypes_change(df):

    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])
    df['GeplantesInbetriebnahmedatum'] = pd.to_datetime(df['GeplantesInbetriebnahmedatum'])
    df['Energietraeger'] = pd.to_numeric(df['Energietraeger'])
    #df['NameStromerzeugungseinheit'] = pd.to_numeric(df['NameStromerzeugungseinheit'])
    #df['Ort'] = pd.to_numeric(df['Ort'])
    df['Notstromaggregat'] = pd.to_numeric(df['Notstromaggregat'])
    df['Breitengrad'] = pd.to_numeric(df['Breitengrad'])
    df['Kraftwerksnummer_nv'] = pd.to_numeric(df['Kraftwerksnummer_nv'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])
    df['Gemeindeschluessel'] = pd.to_numeric(df['Gemeindeschluessel'])
    df['BestandteilGrenzkraftwerk'] = pd.to_numeric(df['BestandteilGrenzkraftwerk'])
    #df['AnlagenbetreiberMastrNummer'] = pd.to_numeric(df['AnlagenbetreiberMastrNummer'])
    df['ZugeordnenteWirkleistungWechselrichter'] = pd.to_numeric(df['ZugeordnenteWirkleistungWechselrichter'])
    df['Land'] = pd.to_numeric(df['Land'])
    #df['WeicDisplayName'] = pd.to_numeric(df['WeicDisplayName'])
    df['AcDcKoppelung'] = pd.to_numeric(df['AcDcKoppelung'])
    df['Laengengrad'] = pd.to_numeric(df['Laengengrad'])
    df['PumpbetriebKontinuierlichRegelbar'] = pd.to_numeric(df['PumpbetriebKontinuierlichRegelbar'])
    #df['GenMastrNummerv'] = pd.to_numeric(df['GenMastrNummer'])
    #df['Gemarkung'] = pd.to_numeric(df['Gemarkung'])
    df['EinheitSystemstatus'] = pd.to_numeric(df['EinheitSystemstatus'])
    df['EinheitBetriebsstatus'] = pd.to_numeric(df['EinheitBetriebsstatus'])
    df['Einspeisungsart'] = pd.to_numeric(df['Einspeisungsart'])
    df['Hausnummer_nv'] = pd.to_numeric(df['Hausnummer_nv'])
    #df['Postleitzahl'] = pd.to_numeric(df['Postleitzahl'])
    df['NichtVorhandenInMigriertenEinheiten'] = pd.to_numeric(df['NichtVorhandenInMigriertenEinheiten'])
    df['Technologie'] = pd.to_numeric(df['Technologie'])
    #df['Kraftwerksnummer'] = pd.to_numeric(df['Kraftwerksnummer'])
    df['Pumpspeichertechnologie'] = pd.to_numeric(df['Pumpspeichertechnologie'])
    df['Batterietechnologie'] = pd.to_numeric(df['Batterietechnologie'])
    df['FernsteuerbarkeitDv'] = pd.to_numeric(df['FernsteuerbarkeitDv'])
    df['NetzbetreiberpruefungStatus'] = pd.to_numeric(df['NetzbetreiberpruefungStatus'])
    df['DatumEndgueltigeStilllegung'] = pd.to_datetime(df['DatumEndgueltigeStilllegung'], errors='coerce')
    df['DatumDesBetreiberwechsels'] = pd.to_datetime(df['DatumDesBetreiberwechsels'])
    df['FernsteuerbarkeitDr'] = pd.to_numeric(df['FernsteuerbarkeitDr'])
    df['DatumBeginnVoruebergehendeStilllegung'] = pd.to_datetime(df['DatumBeginnVoruebergehendeStilllegung'])
    df['NetzbetreiberpruefungDatum'] = pd.to_datetime(df['NetzbetreiberpruefungDatum'])
    #df['Weic'] = pd.to_numeric(df['Weic'])
    #df['EegMaStRNummer'] = pd.to_numeric(df['EegMaStRNummer'])
    #df['SpeMastrNummer'] = pd.to_numeric(df['SpeMastrNummer'])
    df['DatumRegistrierungDesBetreiberwechsels'] = pd.to_datetime(df['DatumRegistrierungDesBetreiberwechsels'])
    df['Bundesland'] = pd.to_numeric(df['Bundesland'])
    df['AnschlussAnHoechstOderHochSpannung'] = pd.to_numeric(df['AnschlussAnHoechstOderHochSpannung'])
    df['Einsatzort'] = pd.to_numeric(df['Einsatzort'])
    df['Einsatzverantwortlicher'] = pd.to_numeric(df['Einsatzverantwortlicher'])
    df['PumpbetriebLeistungsaufnahme'] = pd.to_numeric(df['PumpbetriebLeistungsaufnahme'])
    df['EegAnlagentyp'] = pd.to_numeric(df['EegAnlagentyp'])
    #df['LokationMaStRNummer'] = pd.to_numeric(df['LokationMaStRNummer'])
    #df['FlurFlurstuecknummern'] = pd.to_numeric(df['FlurFlurstuecknummern'])
    df['Inbetriebnahmedatum'] = pd.to_datetime(df['Inbetriebnahmedatum'])
    df['StrasseNichtGefunden'] = pd.to_numeric(df['StrasseNichtGefunden'])
    df['Bruttoleistung'] = pd.to_numeric(df['Bruttoleistung'])
    #df['Adresszusatz'] = pd.to_numeric(df['Adresszusatz'])
    df['Weic_nv'] = pd.to_numeric(df['Weic_nv'])
    #df['Gemeinde'] = pd.to_numeric(df['Gemeinde'])
    #df['Landkreis'] = pd.to_numeric(df['Landkreis'])
    #df['HausnummerNichtGefunden'] = pd.to_numeric(df['HausnummerNichtGefunden'])
    #df['EinheitMastrNummer'] = pd.to_numeric(df['EinheitMastrNummer'])
    df['Nettonennleistung'] = pd.to_numeric(df['Nettonennleistung'])
    #df['Hausnummer'] = pd.to_numeric(df['Hausnummer'])
    df['FernsteuerbarkeitNb'] = pd.to_numeric(df['FernsteuerbarkeitNb'])
    df['DatumWiederaufnahmeBetrieb'] = pd.to_datetime(df['DatumWiederaufnahmeBetrieb'])
    #df['Strasse'] = pd.to_numeric(df['Strasse'])
    df['NettonennleistungDeutschland'] = pd.to_numeric(df['NettonennleistungDeutschland'])


In [23]:

#file 4
def EinheitenSolar_columns():

    my_dict={'Nebenausrichtung':[],
             'Hauptausrichtung':[],
             'Strasse':[],
             'Gemarkung':[],
             'GeplantesInbetriebnahmedatum':[],
             'Nutzungsbereich':[],
             'AnzahlModule':[],
             'EinheitBetriebsstatus':[],
             'Einspeisungsart':[],
             'ZugeordneteWirkleistungWechselrichter':[],
             'InAnspruchGenommeneFlaeche':[],
             'Gemeindeschluessel':[],
             'LokationMaStRNummer':[],
             'AnschlussAnHoechstOderHochSpannung':[],
             'DatumWiederaufnahmeBetrieb':[],
             'GenMastrNummer':[],
             'Gemeinde':[],
             'DatumEndgueltigeStilllegung':[],
             'NameStromerzeugungseinheit':[],
             'FernsteuerbarkeitNb':[],
             'DatumDesBetreiberwechsels':[],
             'Laengengrad':[],
             'Breitengrad':[],
             'EinheitMastrNummer':[],
             'Energietraeger':[],
             'Lage':[],
             'StrasseNichtGefunden':[],
             'Postleitzahl':[],
             'Landkreis':[],
             'Leistungsbegrenzung':[],
             'Adresszusatz':[],
             'ArtDerFlaecheIds':[],
             'Ort':[],
             'NetzbetreiberpruefungDatum':[],
             'AnlagenbetreiberMastrNummer':[],
             'Weic_nv':[],
             'GemeinsamerWechselrichterMitSpeicher':[],
             'Registrierungsdatum':[],
             'HauptausrichtungNeigungswinkel':[],
             'Hausnummer':[],
             'FernsteuerbarkeitDv':[],
             'NebenausrichtungNeigungswinkel':[],
             'EinheitlicheAusrichtungUndNeigungswinkel':[],
             'InAnspruchGenommeneAckerflaeche':[],
             'HausnummerNichtGefunden':[],
             'DatumBeginnVoruebergehendeStilllegung':[],
             'EegMaStRNummer':[],
             'NetzbetreiberpruefungStatus':[],
             'Bundesland':[],
             'EinheitSystemstatus':[],
             'Hausnummer_nv':[],
             'DatumRegistrierungDesBetreiberwechsels':[],
             'FernsteuerbarkeitDr':[],
             'Einsatzverantwortlicher':[],
             'DatumLetzteAktualisierung':[],
             'FlurFlurstuecknummern':[],
             'NichtVorhandenInMigriertenEinheiten':[],
             'Kraftwerksnummer_nv':[],
             'Land':[],
             'Nettonennleistung':[],
             'Bruttoleistung':[],
             'Inbetriebnahmedatum':[]}
    return my_dict

def EinheitenSolar_datatypes_change(df):

    df['Nebenausrichtung'] = pd.to_numeric(df['Nebenausrichtung'])
    df['Hauptausrichtung'] = pd.to_numeric(df['Hauptausrichtung'])
    #df['Strasse'] = pd.to_numeric(df['Strasse'])
    #df['Gemarkung'] = pd.to_numeric(df['Gemarkung'])
    df['GeplantesInbetriebnahmedatum'] = pd.to_datetime(df['GeplantesInbetriebnahmedatum'])
    df['Nutzungsbereich'] = pd.to_numeric(df['Nutzungsbereich'])
    df['AnzahlModule'] = pd.to_numeric(df['AnzahlModule'])
    df['EinheitBetriebsstatus'] = pd.to_numeric(df['EinheitBetriebsstatus'])
    df['Einspeisungsart'] = pd.to_numeric(df['Einspeisungsart'])
    df['ZugeordneteWirkleistungWechselrichter'] = pd.to_numeric(df['ZugeordneteWirkleistungWechselrichter'])
    df['InAnspruchGenommeneFlaeche'] = pd.to_numeric(df['InAnspruchGenommeneFlaeche'])
    df['Gemeindeschluessel'] = pd.to_numeric(df['Gemeindeschluessel'])
    #df['LokationMaStRNummer'] = pd.to_numeric(df['LokationMaStRNummer'])
    #df['AnschlussAnHoechstOderHochSpannung'] = pd.to_numeric(df['AnschlussAnHoechstOderHochSpannung'])
    df['DatumWiederaufnahmeBetrieb'] = pd.to_datetime(df['DatumWiederaufnahmeBetrieb'])
    #df['GenMastrNummer'] = pd.to_numeric(df['GenMastrNummer'])
    #df['Gemeinde'] = pd.to_numeric(df['Gemeinde'])
    df['DatumEndgueltigeStilllegung'] = pd.to_datetime(df['DatumEndgueltigeStilllegung'])
    #df['NameStromerzeugungseinheit'] = pd.to_numeric(df['NameStromerzeugungseinheit'])
    df['FernsteuerbarkeitNb'] = pd.to_numeric(df['FernsteuerbarkeitNb'])
    df['DatumDesBetreiberwechsels'] = pd.to_datetime(df['DatumDesBetreiberwechsels'])
    df['Laengengrad'] = pd.to_numeric(df['Laengengrad'])
    df['Breitengrad'] = pd.to_numeric(df['Breitengrad'])
    #df['EinheitMastrNummer'] = pd.to_numeric(df['EinheitMastrNummer'])
    df['Energietraeger'] = pd.to_numeric(df['Energietraeger'])
    df['Lage'] = pd.to_numeric(df['Lage'])
    df['StrasseNichtGefunden'] = pd.to_numeric(df['StrasseNichtGefunden'])
    df['Postleitzahl'] = pd.to_numeric(df['Postleitzahl'])
    #df['Landkreis'] = pd.to_numeric(df['Landkreis'])
    df['Leistungsbegrenzung'] = pd.to_numeric(df['Leistungsbegrenzung'])
    #df['Adresszusatz'] = pd.to_numeric(df['Adresszusatz'])
    #df['ArtDerFlaecheIds'] = pd.to_numeric(df['ArtDerFlaecheIds'])
    #df['Ort'] = pd.to_numeric(df['Ort'])
    df['NetzbetreiberpruefungDatum'] = pd.to_datetime(df['NetzbetreiberpruefungDatum'])
    #df['AnlagenbetreiberMastrNummer'] = pd.to_numeric(df['AnlagenbetreiberMastrNummer'])
    df['Weic_nv'] = pd.to_numeric(df['Weic_nv'])
    df['GemeinsamerWechselrichterMitSpeicher'] = pd.to_numeric(df['GemeinsamerWechselrichterMitSpeicher'])
    df['Registrierungsdatum'] = pd.to_datetime(df['Registrierungsdatum'])
    df['HauptausrichtungNeigungswinkel'] = pd.to_numeric(df['HauptausrichtungNeigungswinkel'])
    #df['Hausnummer'] = pd.to_numeric(df['Hausnummer'])
    df['FernsteuerbarkeitDv'] = pd.to_numeric(df['FernsteuerbarkeitDv'])
    df['NebenausrichtungNeigungswinkel'] = pd.to_numeric(df['NebenausrichtungNeigungswinkel'])
    df['EinheitlicheAusrichtungUndNeigungswinkel'] = pd.to_numeric(df['EinheitlicheAusrichtungUndNeigungswinkel'])
    df['InAnspruchGenommeneAckerflaeche'] = pd.to_numeric(df['InAnspruchGenommeneAckerflaeche'])
    df['HausnummerNichtGefunden'] = pd.to_numeric(df['HausnummerNichtGefunden'])
    df['DatumBeginnVoruebergehendeStilllegung'] = pd.to_datetime(df['DatumBeginnVoruebergehendeStilllegung'])
    #df['EegMaStRNummer'] = pd.to_numeric(df['EegMaStRNummer'])
    df['NetzbetreiberpruefungStatus'] = pd.to_numeric(df['NetzbetreiberpruefungStatus'])
    df['Bundesland'] = pd.to_numeric(df['Bundesland'])
    df['EinheitSystemstatus'] = pd.to_numeric(df['EinheitSystemstatus'])
    df['Hausnummer_nv'] = pd.to_numeric(df['Hausnummer_nv'])
    df['DatumRegistrierungDesBetreiberwechsels'] = pd.to_datetime(df['DatumRegistrierungDesBetreiberwechsels'])
    df['FernsteuerbarkeitDr'] = pd.to_numeric(df['FernsteuerbarkeitDr'])
    df['Einsatzverantwortlicher'] = pd.to_numeric(df['Einsatzverantwortlicher'])
    df['DatumLetzteAktualisierung'] = pd.to_datetime(df['DatumLetzteAktualisierung'])
    #df['FlurFlurstuecknummern'] = pd.to_numeric(df['FlurFlurstuecknummern'])
    df['NichtVorhandenInMigriertenEinheiten'] = pd.to_numeric(df['NichtVorhandenInMigriertenEinheiten'])
    df['Kraftwerksnummer_nv'] = pd.to_numeric(df['Kraftwerksnummer_nv'])
    df['Land'] = pd.to_numeric(df['Land'])
    df['Nettonennleistung'] = pd.to_numeric(df['Nettonennleistung'])
    df['Bruttoleistung'] = pd.to_numeric(df['Bruttoleistung'])
    df['Inbetriebnahmedatum'] = pd.to_datetime(df['Inbetriebnahmedatum'])


In [24]:

# for processing all the xml files in the dataset folder
def names_of_xml_files(dataset_path):
    '''

    :param dataset_path: of zip file
    :return: final list of files
    '''
    try:

        zf = zipfile.ZipFile(dataset_path, "r")

        xml_files = []
        for file in zf.namelist():
            if file.endswith(".xml"):
                xml_files.append(os.path.join(file))

        # print(xml_files)

        new_list = []
        for file in xml_files:
            file = file[:-4]
            new_list.append(file)

        list_changed = [f.replace("_", "") for f in new_list]
        for i in range(0, 10):
            list_changed = [f.replace(str(i), "") for f in list_changed]

        final_list_of_files = list(set(list_changed))
        return final_list_of_files
    except Exception as e:
        print("Error while capturing file names :", e)
        pass

In [25]:


# give path of dataset and file name without the number
def files_mask(dataset_path, filename):
    '''
    
    :param dataset_path: provide dataset path of zip folder
    :param filename: file name without extension type
    :return: matching list with the same kind of files
    '''
    # get all files names in the datasetfolder

    zf = zipfile.ZipFile(dataset_path, "r")
    all_files = []
    for file in zf.namelist():
        if file.endswith(".xml"):
            #[5:] removes DATA/ string from the beginning
            all_files.append(file)

    #all_files = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]

    regex_pattern = ".*" + filename + ".*"

    match_list = []
    for i in range(0, len(all_files)):
        match = re.findall(regex_pattern, all_files[i])
        if not match:
            # if list is empty
            pass
        else:
            match_list.append(match[0])

    # print("Required Files in the Dataset: ", match_list)
    return match_list



In [26]:


def xml_to_dataframe_converter(dataset_path, xml_file):
    '''
    
    :param dataset_path:zip folder path 
    :param xml_file: full file name with the extension
    :return: 
    '''

    zf = zipfile.ZipFile(dataset_path, "r")

    #if xml_file in zf.namelist():
    xml_file_open = zf.open(xml_file)
    tree = ET.parse(xml_file_open)
    root = tree.getroot()

    # defining the dictionary from respective files
    if ("AnlagenEegSolar" in xml_file):
        my_dict = AnlagenEegSolar_columns()
    elif ("AnlagenEegSpeicher" in xml_file):
        my_dict = AnlagenEegSpeicher_columns()
    elif ("AnlagenStromSpeicher" in xml_file):
        my_dict = AnlagenStromSpeicher_columns()
    elif ("EinheitenStromSpeicher" in xml_file):
        my_dict = EinheitenStromSpeicher_columns()
    elif ("EinheitenSolar" in xml_file):
        my_dict = EinheitenSolar_columns()
    else:
        print("Could not find the file specified.")

    for elem in root:
        for sub_elem in elem:
            # print(sub_elem.tag," : " ,sub_elem.text)
            # check if key already exists in the dictionary
            if my_dict.__contains__(sub_elem.tag):
                my_dict[sub_elem.tag].append(sub_elem.text)
            # else create the new key and append
            else:
                # my_dict[sub_elem.tag] = []
                # my_dict[sub_elem.tag].append(sub_elem.text)
                print("Add this column: ",my_dict[sub_elem.tag])

    df = pd.DataFrame.from_dict(my_dict, orient='index').transpose()

    #changing column data types by calling the function
    if ("AnlagenEegSolar" in xml_file):
        AnlagenEegSolar_datatypes_change(df)
    elif ("AnlagenEegSpeicher" in xml_file):
        AnlagenEegSpeicher_datatypes_change(df)
    elif ("AnlagenStromSpeicher" in xml_file):
        AnlagenStromSpeicher_datatypes_change(df)
    elif ("EinheitenStromSpeicher" in xml_file):
        EinheitenStromSpeicher_datatypes_change(df)
    elif ("EinheitenSolar" in xml_file):
        EinheitenSolar_datatypes_change(df)

    #print(df.dtypes)
    return df

In [27]:
import warnings
warnings.filterwarnings('ignore') 

#new main
def main():
    # change dataset path
    #dataset_path = "G:\DATA GURUS WORK\AAAA\"
    dataset_path = "/home/jovyan/work/AAAA.zip"

    file_names = ['AnlagenEegSolar', 'AnlagenEegSpeicher', 'AnlagenStromSpeicher', 'EinheitenStromSpeicher', 'EinheitenSolar']


    #if uncommmented then add all the below lines under this loop, before the conn.close
    for i in range(0, len(file_names)):
    
        file_name = file_names[i]


        print("Current File Type : ", file_name)

        xml_files_list = files_mask(dataset_path, file_name)
        print("XML FILES LIST: ", xml_files_list)

        for xml_file in xml_files_list:

            print(f"Processing : {xml_file}")
            data = xml_to_dataframe_converter(dataset_path,xml_file)
            # Create DataFrame
            df = pd.DataFrame(data)
            #creating spark data frame
            sdf = spark.createDataFrame(df)


            # change database table name
            table_name = file_name.lower()
            table_name = "marktstammdaten_"+table_name

            try:
                # df.to_sql(table_name, con=conn, if_exists='append', index=False)
                sdf.write.saveAsTable(table_name)
                print("Done")
            except Exception as e:
                # print("Issue : ", e)
                # print("creating new column and re-writing the data")
                # sql_string = '''SELECT * FROM {}'''.format(table_name)
                # old_data = pd.read_sql(sql_string, conn)
                # df_new = pd.concat([old_data, df])
                # df_new.to_sql(table_name, con=conn, if_exists='replace', index=False)
                print("Done")

    #conn.close()
    

In [28]:

if __name__ == "__main__":
    #main()
    main()


Current File Type :  AnlagenEegSolar
XML FILES LIST:  ['AAAA/AnlagenEegSolar_1.xml', 'AAAA/AnlagenEegSolar_2.xml']
Processing : AAAA/AnlagenEegSolar_1.xml


/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Done
Processing : AAAA/AnlagenEegSolar_2.xml


/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Done
Current File Type :  AnlagenEegSpeicher
XML FILES LIST:  ['AAAA/AnlagenEegSpeicher_2.xml', 'AAAA/AnlagenEegSpeicher_3.xml', 'AAAA/AnlagenEegSpeicher_4.xml', 'AAAA/AnlagenEegSpeicher_5.xml']
Processing : AAAA/AnlagenEegSpeicher_2.xml


...

Done
Processing : AAAA/EinheitenStromSpeicher_2.xml


/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Done
Current File Type :  EinheitenSolar
XML FILES LIST:  ['AAAA/EinheitenSolar_26.xml']
Processing : AAAA/EinheitenSolar_26.xml


/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Done


In [29]:
new_df = spark.read.table("marktstammdaten_anlageneegspeicher")

new_df.show

<bound method DataFrame.show of DataFrame[EegMaStRNummer: string, Registrierungsdatum: timestamp, EegInbetriebnahmedatum: timestamp, VerknuepfteEinheitenMaStRNummern: string, DatumLetzteAktualisierung: timestamp]>